*This notebook provide the best practices to consume covid-lake API.*

*For more details, please check the swagger documentation*

In [11]:
# General Import
import json
import requests 
import pandas as pd

In [12]:
def func(URL, PARAMS=''):
    """
    Loads data into a pandas dataframe
    
    Parameter
    ---------
    URL: api url to invoke
    PARAMS: include in the request
    
    Return
    ------
    dataframe
    """
    res = requests.get(url=URL, params=PARAMS)
    res_json = json.loads(res.content)
    if res.status_code == 200:
        return pd.DataFrame(res_json)
    else:
        raise ValueError(res_json.get('error'))

### API 1: showTables

Presents an overview of the tables existing in the covid-lake.

URL: http://23.20.157.201/api/showTables

In [3]:
URL = "http://23.20.157.201/api/showTables"
df = func(URL)

In [4]:
df.head()

,TABLE_NAME,TABLE_SCHEMA
0,Institution,Covid-Lake
1,Socioeconomic_Factors,Covid-Lake
2,Disease_Indicator,Covid-Lake
3,News&Trends,Covid-Lake


### API 2: getTable

Serves to load a given table.

URL: http://23.20.157.201/api/getTable

PARAMS: name of the table

In [7]:
URL = "http://23.20.157.201/api/getTable"

PARAMS = {'table_name': 'Disease_Indicator'} 

df = func(URL, PARAMS)

In [8]:
df.head()

,addressCity,addressCountry,addressState,addressType_display,createdOn,identifier,valueString,valueType_display
0,,,RO,,2020-01-30,health_ministry,0,new_cases
1,,,AC,,2020-01-30,health_ministry,0,new_cases
2,,,AM,,2020-01-30,health_ministry,0,new_cases
3,,,RR,,2020-01-30,health_ministry,0,new_cases
4,,,PA,,2020-01-30,health_ministry,0,new_cases


In [7]:
df.shape

(5, 3)

### API 3: querySQL

Runs personalized queries.

**NB: you are allowed to run only SELECT queries**

URL: http://23.20.157.201/api/querySQL

PARAMS: query

In [13]:
def func(URL, PARAMS=''):
    """
    Loads data into a pandas dataframe
    
    Parameter
    ---------
    URL: api url to invoke
    PARAMS: include in the request
    
    Return
    ------
    dataframe
    """
    res = requests.get(url=URL, params=PARAMS)
    print("passou aqui")
    print(res.url)
    res_json = json.loads(res.content)
    if res.status_code == 200:
        return pd.DataFrame(res_json)
    else:
        raise ValueError(res_json.get('error'))
        
import requests 
        
URL = "http://18.207.22.2/api/querySQL"

query = """SELECT * FROM "Covid-Lake"."Disease_Indicator" ORDER BY "createdOn" LIMIT 10"""

PARAMS = {'query': query} 
df = func(URL, PARAMS)

passou aqui
http://18.207.22.2/api/querySQL?query=SELECT+%2A+FROM+%22Covid-Lake%22.%22Disease_Indicator%22+ORDER+BY+%22createdOn%22+LIMIT+10


In [8]:
df.head()

,addressCity,addressCountry,addressState,addressType_display,createdOn,identifier,valueString,valueType_display
0,,,AC,,2020-01-30,health_ministry,0,new_cases
1,,,AM,,2020-01-30,health_ministry,0,new_cases
2,,,RR,,2020-01-30,health_ministry,0,new_cases
3,,,PA,,2020-01-30,health_ministry,0,new_cases
4,,,AP,,2020-01-30,health_ministry,0,new_cases


In [23]:
df.shape

(15, 3)

In [10]:
URL = "http://23.20.157.201/api/querySQL"

query = """DROP "Covid-Lake"."News&Trends" """

PARAMS = {'query': query} 

df = func(URL, PARAMS)

ValueError: 400 Bad Request: DROP: Permission denied.

**Developed by Datasprints**

In [ ]:
@app.route("/getNews-Trends", methods=['GET'])
def getNewsTrends():
    import ctypes
    file = open('news-trends.txt') 
    df_id = file.read()
    if df_id:
        print("exist")
        print(ctypes.cast(int(df_id), ctypes.py_object).value)
        return ctypes.cast(int(df_id), ctypes.py_object).value
    else:
        print("First visit")
        table_name = "News&Trends"
        disease_df = dremio_conn.getTable(table_name)
        # Write the id
        with open('news-trends.txt', 'w') as f:
            f.write(str(id(disease_df)))
        response = make_response(disease_df.to_csv())
        response.headers["Content-Disposition"] = "attachment; filename={}.csv".format(table_name)
        response.headers["Content-Type"] = "text/csv"
        return response